# Lerdahlの和音距離を実装

## F. Lerdahl & C. L. Krumhansl (2007). "Modeling Tonal Tension". *Music Perception*. vol24-4, 329-366. 

In [1]:
class Note:
    def __init__(self):
        self.notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
        self.len_notes = len(self.notes)
        
    def enharmonic(self, note):
        
        '''
        returns the enharmonic of a note/key, converts every note in sharp base
        '''
        
        if note == 'Db':
            return 'C#'
        elif note == 'Eb':
            return 'D#'
        elif note == 'E#':
            return 'F'
        elif note == 'Fb':
            return 'E'
        elif note == 'Gb':
            return 'F#'
        elif note == 'Ab':
            return 'G#'
        elif note == 'Bb':
            return 'A#'
        elif note == 'B#':
            return 'C'
        elif note == 'Cb':
            return 'B'
        elif note.istitle():
            # returns a major key
            return note
        elif len(note) == 1:
            # returns a minor key that does not need to be enharmonized
            return note
        else:
            # minor key that has to be capitalized
            note = note.capitalize()
            return note.lower()

In [2]:
class Scale(Note):
    
    def __init__(self):
        
        super().__init__()

    def getDiatonic(self, root, style='natural'):
        '''
        `root` should be a string, the root note of a chord you desire to have.
        An upper case letter will represent a major scale, and a lower case letter will represent a minor scale.
        This function returns a string array of chords, for example, ['C', 'D', 'E', 'F', 'G', 'A', 'B']
        '''        
        major = False
        if self.isMajor(root):
            major = True
        
        root = root.capitalize()
        
        root = self.enharmonic(root)
            
        idx_root = self.notes.index(root)
        
        idx_2 = (idx_root + 2) % self.len_notes
        idx_4 = (idx_root + 5) % self.len_notes
        idx_5 = (idx_root + 7) % self.len_notes
        
        
        if major:
            idx_3 = (idx_root + 4)  % self.len_notes
            idx_6 = (idx_root + 9)  % self.len_notes
            idx_7 = (idx_root + 11) % self.len_notes
        else:
            idx_3 = (idx_root + 3)  % self.len_notes
            if style=='natural':
                idx_6 = (idx_root + 8)  % self.len_notes
                idx_7 = (idx_root + 10) % self.len_notes
            elif style=='harmonic':
                idx_6 = (idx_root + 8)  % self.len_notes
                idx_7 = (idx_root + 11) % self.len_notes                
            elif style=='melodic':
                idx_6 = (idx_root + 9)  % self.len_notes
                idx_7 = (idx_root + 11) % self.len_notes                
        
        diatonic = [root, 
                    self.notes[idx_2], 
                    self.notes[idx_3], 
                    self.notes[idx_4], 
                    self.notes[idx_5], 
                    self.notes[idx_6], 
                    self.notes[idx_7]]
        
        return diatonic
        
    def getChromatic(self, root):
        '''
        `root` should be a string, the root note of a chord you desire to have.
        The case of a letter does not matter.
        This function returns a string array of chords, for example, ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
        '''        
        root = root.capitalize()
        
        root = self.enharmonic(root)
        
        idx_root = self.notes.index(root)
            
        chromatic = self.notes[idx_root:-1] + self.notes[0:idx_root]

        return chromatic
    
    def moveCircle(self, key1, key2):
        '''
        calcultes the number of moves from key1 to key2 in the circle of fifth
        '''
        key1 = self.enharmonic(key1)
        key2 = self.enharmonic(key2)
        
        # calculate on the major side of the circle
        if not self.isMajor(key1):
            key1 = self.getParallel(key1)
        if not self.isMajor(key2):
            key2 = self.getParallel(key2)
        
        key1_up   = key1
        key1_down = key1
                
        idx_key1_up   = self.notes.index(key1)
        idx_key1_down = self.notes.index(key1)
        idx_key2      = self.notes.index(key2)
        
        n_moves = 0
        
        while key1_up!=key2 and key1_down!=key2:
            idx_key1_up   = (idx_key1_up   + 5) % self.len_notes
            idx_key1_down = (idx_key1_down - 5) % self.len_notes
            key1_up       = self.notes[idx_key1_up]
            key1_down     = self.notes[idx_key1_down]
            
            n_moves += 1
        
        return n_moves
    
    def getParallel(self, key):
        '''
        Returns a parallel key, input must be a string.
        getParallel('C') returns 'a'
        getParallel('e') returns 'G'
        '''
        
        if self.isMajor(key):
            key = self.enharmonic(key)
            idx_key = self.notes.index(key)
            idx_parallel_key = (idx_key - 3) % 12
        
        else:
            key = key.capitalize()
            key = self.enharmonic(key)
            idx_key = self.notes.index(key)
            idx_parallel_key = (idx_key + 3) % 12
        
        parallel_key = self.notes[idx_parallel_key]

        return parallel_key
        
    def isMajor(self, key):
        '''
        Returns true if an input key is a major key. 
        '''
        if key.istitle():
            return True
        return False

In [3]:
class Chord(Scale):

    '''
    This class treats all notes in sharps
    '''
    def __init__(self):
        
        super().__init__()
        
    def getDist(self, chord1, chord2, key1=None, key2=None):
        '''
        calculate chord distance
        '''
        if not key1:
            key1 = chord1[0]
        if not key2:
            key2 = chord2[0]
        
        # calculate i: number of moves on the cycle of fifths in the level of diatonic collection (tonic key)
        i = self.moveCircle(key1, key2)
        
        # calculate j: number of moves on the cycle of fifths in the level of chords
        j = self.moveCircle(chord1[0], chord2[0])
        
        # calculate k: number of uncommon notes
        k = 0
        
        # root level
        roots1 = [chord1[0]]
        roots2 = [chord2[0]]
        k += len(roots1) - len(set(roots1) & set(roots2))
        
        # fifths level
        fifths1 = [chord1[0], chord1[2]]
        fifths2 = [chord2[0], chord2[2]]
        k += len(fifths1) - len(set(fifths1) & set(fifths2))        
        
        # triadic level
        triads1 = [chord1[0], chord1[1], chord1[2]]
        triads2 = [chord2[0], chord2[1], chord2[2]]
        k += len(triads1) - len(set(triads1) & set(triads2))      
        
        # diatonic level
        diatonic1 = self.getDiatonic(key1)
        diatonic2 = self.getDiatonic(key2)
        k += len(diatonic1) - len(set(diatonic1) & set(diatonic2))      
        
        # chromatic level
        chromatic1 = self.getChromatic(key1)
        chromatic2 = self.getChromatic(key2)
        k += len(chromatic1) - len(set(chromatic1) & set(chromatic2))    
        
        print("i: ", i)
        print("j: ", j)
        print("k: ", k)
        
        delta = i + j + k
        
        return delta
        
    def getChord(self, root, tension=''):
        '''
        `root` should be a string, the root note of a chord you desire to have. 
        An upper case letter will represent a major chord, and a lower case letter will represent a minor chord.
        This function returns a string array of chords, for example, ['C', 'E', 'G']
        As for 20190419, only major or minor chords could be returned, but other tension chords will be added in the future update.
        '''
        major = False
        if self.isMajor(root):
            major = True
        root = root.capitalize()
        
        root = self.enharmonic(root)
            
        idx_root = self.notes.index(root)
        
        if major:
            idx_3 = (idx_root + 4) % self.len_notes
            idx_5 = (idx_root + 7) % self.len_notes
        else:
            idx_3 = (idx_root + 3) % self.len_notes
            idx_5 = (idx_root + 7) % self.len_notes
        
        # special chords
        if 'sus4' in tension:
            idx_3 = (idx_root + 5) % self.len_notes
        elif 'dim' in tension:
            idx_3 = (idx_root + 3) % self.len_notes
            idx_5 = (idx_root + 6) % self.len_notes
        elif 'aug' in tension:
            idx_3 = (idx_root + 4) % self.len_notes            
            idx_5 = (idx_root + 8) % self.len_notes
        
        chord = [root, 
                 self.notes[idx_3], 
                 self.notes[idx_5]]
        
        # add tension
        idx_ts = list()
        
        if '6' in tension:
            idx_ts.append((idx_root + 9) % self.len_notes)
            
        if 'M' in tension: # M7, but using this since M9 counts
            idx_ts.append((idx_root + 11) % self.len_notes)
        elif '7' in tension:
            idx_ts.append((idx_root + 10) % self.len_notes)
            
        if 'add9' in tension:
            idx_ts = [((idx_root + 14) % self.len_notes)]
        elif 'b9' in tension:
            idx_ts.append((idx_root + 13) % self.len_notes)
        elif '#9' in tension:
            idx_ts.append((idx_root + 15) % self.len_notes)
        elif '9' in tension:
            idx_ts.append((idx_root + 14) % self.len_notes)
            
        if '#11' in tension:
            idx_ts.append((idx_root + 18) % self.len_notes)
        elif '11' in tension:
            idx_ts.append((idx_root + 17) % self.len_notes)       
            
        if 'b13' in tension:
            idx_ts.append((idx_root + 22) % self.len_notes)        
        elif '13' in tension:
            idx_ts.append((idx_root + 22) % self.len_notes)            
            
        for idx_t in idx_ts:
            chord.append(self.notes[idx_t])
            
        return chord
    

## Examples

### Get a diatonic scale of B flat minor

In [4]:
Scale().getChromatic('bb')

['A#', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A']

### Get the distance between C chord on C key and G chord on C key

In [5]:
chord1 = Chord().getChord('C')
chord2 = Chord().getChord('G')
print(chord1)
Chord().getDist(chord1, chord2, key1='C', key2='C')

['C', 'E', 'G']
i:  0
j:  1
k:  4


5

### Get the distance between C chord on C key and G chord on G key

In [12]:
chord1 = Chord().getChord('C')
chord2 = Chord().getChord('G')
Chord().getDist(chord1, chord2, key1='C', key2='G')

i:  1
j:  1
k:  5


7

### Get the distance between C chord on C key and G chord on C key

In [18]:
chord1 = Chord().getChord('C')
chord2 = Chord().getChord('G')
Chord().getDist(chord1, chord2, key1='C', key2='C')

i:  0
j:  1
k:  4


5

### Get the distance between C chord on C key and C minor chord on C minor key

In [14]:
chord1 = Chord().getChord('C')
chord2 = Chord().getChord('c')
Chord().getDist(chord1, chord2, key1='C', key2='c')

i:  3
j:  0
k:  4


7